In [1]:
import cv2
import numpy as np
import os
import h5py
import pandas as pd
from PIL import Image
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
from keras.preprocessing import image

In [2]:
train=pd.read_csv('train_new.csv')

In [3]:
train.tail()

,image,class
99194,V995.png,V
99195,V996.png,V
99196,V997.png,V
99197,V998.png,V
99198,V999.png,V


In [4]:
train['class'].value_counts()

N    75709
M     8405
Q     6431
V     5789
S     2223
F      642
Name: class, dtype: int64

In [8]:
dataframe = pd.DataFrame(train)
dataframe.rename(columns={'class': 'Label'}, inplace=True)
dataframe

,image,Label
0,F1.png,F
1,F10.png,F
2,F100.png,F
3,F101.png,F
4,F102.png,F
...,...,...
99194,V995.png,V
99195,V996.png,V
99196,V997.png,V
99197,V998.png,V


In [9]:
samples = []
for category in ['N','M','Q','V','S','F']:
    category_slice = train.query("Label == @category")
    samples.append(category_slice.sample(640, random_state=1))

dataframe_train = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
dataframe_train['Label'].value_counts()

N    640
Q    640
S    640
F    640
M    640
V    640
Name: Label, dtype: int64

In [ ]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = Image.open('input/train/'+train['class'][i]+'/'+train['image'][i])
    target_size=(64,64)
    img = img.resize(target_size)
    #convert image to numpy array
    img_array = np.array(img)
    img_array = img_array/255
    train_image.append(img)
X = np.array(train_image)

In [ ]:
y=train['class']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [ ]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
# normalize data
x_train = X_train.astype('float32')
x_test = X_test.astype('float32')

x_train /= 127.5
x_test /= 127.5

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 3),activation='relu', padding='same', input_shape=(64,64,3)))
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(
    loss= 'categorical_crossentropy', 
    optimizer=Adam(lr=0.001, decay=1e-04),
    metrics=['accuracy']
)

In [ ]:
model.summary()

In [ ]:
model.fit(
    X_train,
    y_train,
    batch_size=100,
    epochs=5,
    validation_data=(X_test, y_test),
    shuffle=True
)


In [ ]:
y_pred = model.predict(X_test, batch_size=10, verbose=0)

In [ ]:
y_predict = []
for i in range(0, len(y_pred)):
    y_predict.append(int(np.argmax(y_pred[i])))
len(y_predict)

In [ ]:
y_test

In [ ]:
y_true = []
y_test1 = np.array(y_test)
for i in range(0, len(y_test1)):
    y_true.append(int(np.argmax(y_test1[i])))
len(y_true)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.figure(figsize=(7,7))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes, rotation=45)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
class_names =['Arson','Burglary','Explosion','Fighting','Normal']

In [ ]:
cm_plot_labels = class_names

In [ ]:
l=len(y_true)
for i in range(l):
    print(y_true[i],'----',y_predict[i])

In [ ]:
print(classification_report(y_true=y_true,y_pred=y_predict))

In [ ]:
# to plot confusion matrix
cm = confusion_matrix(y_true=y_true, y_pred=y_predict)
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
acc_rf = accuracy_score(y_true=y_true, y_pred= y_predict)
print("Overall accuracy of CNN model using test-set is : %f" %(acc_rf*100))

In [ ]:
model_structure = model.to_json()
f = Path("models/model_structure.json")
f.write_text(model_structure)

model.save_weights("models/model_weights.h5")

In [ ]:
model.save_weights("models/model_weights.h5")